In [ ]:
import json
import torch
import pandas as pd

from tqdm import tqdm
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM

### Tokenizer and Model

In [3]:
model_id = "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit"

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

In [6]:
model.config

LlamaConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "unsloth/DeepSeek-R1-Distill-Llama-8B-unsloth-bnb-4bit",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": [
      "lm_head",
   

### Data

In [7]:
DATA_DIR = Path.cwd() / "am_reasoning" / "data_files"

In [8]:
df = pd.read_csv((DATA_DIR / "pe_dataset.csv"), index_col=0)

In [9]:
df.columns

Index(['split', 'title', 'argument_bound_1', 'argument_bound_2', 'argument_id',
       'ac', 'label', 'essay'],
      dtype='object')

In [10]:
df.shape

(6089, 8)

In [11]:
df.split.value_counts()

split
TRAIN    4823
TEST     1266
Name: count, dtype: int64

### Template Messages

In [12]:
# sys_msg = """You are an expert in argumentation analysis. You will be given a complete argument. Your task is to analyze the argument holistically and classify it as exactly ONE of the following categories: Position, Claim, or Premise. 

# First, think through your analysis step by step, considering the function and structure of the argument. Explain your reasoning, identifying key characteristics that point to a specific classification.

# Then, provide your final classification as a single word: Position, Claim, or Premise."""

In [13]:
# instruction = """### You are an expert in Argument Mining. You are given an argument component from an essay. Your task is to classify the argument component as either "MajorClaim", "Claim" or "Premise". You must return the argument component type in following JSON format:: {"component_type": [component_type (str)]}


# """

In [14]:
# instruction = """### You are an expert in Argument Mining and discourse analysis. 

# INPUT:
# - You are given an argument component from an essay. 
# - You are given the full text of the essay.

# TASK:
# - Your task is to classify the whole argument component, as ONLY ONE of: "MajorClaim", "Claim" or "Premise".
# - Explain your reasoning step-by-step before making your final classification.

# Then, return your final classification in the following JSON format:
# {"component_type": [Answer]}

# Example:

# {"component_type": ["Claim"]}

# """

In [15]:
reasons_d = {}

In [16]:
instruction = """### You are an expert in Argument Mining and discourse analysis. 

INPUT:
- You are given a single argument component from an essay. 
- You are also given the full text of the essay for context.

TASK:
- Your task is to classify the ENTIRE argument component as ONE of these three categories ONLY: "MajorClaim", "Claim" or "Premise".
- Do NOT break down the argument component into smaller parts with different classifications.
- The WHOLE component must receive exactly ONE classification.

REASONING PROCESS:
First, analyze the component considering these factors:
1. Semantic content: What is the meaning and purpose of this component in the argument?
2. Syntactic structure: How is this component phrased? (declarative statement, supporting evidence, etc.)
3. Context: How does this component relate to the overall essay thesis?
4. Function: 
   - Does it express the author's central position on the main topic? (MajorClaim)
   - Does it express a specific stance that supports or develops the main position? (Claim)
   - Does it provide evidence, examples, or justification for a claim? (Premise)

Provide your step-by-step reasoning, then classify the ENTIRE argument component as exactly ONE of: "MajorClaim", "Claim", or "Premise".

Finally, return your classification in the following JSON format:
{"component_type": ["Your Answer"]}

Example:
{"component_type": ["Claim"]}

"""

In [17]:
df_s = df.sample(20).reset_index(drop=True)

In [18]:
sys_msgs = []
user_msgs = []

for _, row in df_s.iterrows():

    sys_msg = {"role": "system", "content": instruction}
    #usr_msg = {"role": "user", "content": f"""### Here is the argument component to be classified: {row['ac']}"""} 
    
    #msg_c = f"""### Here is the complete essay text: {df.iloc[0]['essay']}""" + "\n\n" + f"""### Here is the argument component to be classified: {row['ac']}"""
    
    usr_msg = {"role": "user", "content": f"""### Here is the complete essay text: {row['essay']}""" + "\n\n" + f"""### Here is the argument component to be classified: {row['ac']}"""}
    
    
    sys_msgs.append(sys_msg)
    user_msgs.append(usr_msg)    
    

In [19]:
messages = []

for i in range(len(sys_msgs)):
    messages.append([sys_msgs[i], user_msgs[i]])

In [20]:
len(messages)

20

In [21]:
messages[5][1]['content'].split("###")[2]

' Here is the argument component to be classified: some lecturers in our university read the same text in the reference books for that particular course module'

In [22]:
#print(usr_msg)

In [23]:
# messages = [
#     {"role": "system", "content": instruction},
#     {"role": "user", "content": usr_msg},
# ]

In [24]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [25]:
for i, message in tqdm(enumerate(messages), "generating CoT", len(messages)):

    input_ids = tokenizer.apply_chat_template(message, add_generation_prompt=True, return_tensors="pt").to(model.device) # type: ignore
    outputs = model.generate(input_ids, max_new_tokens=1024, eos_token_id=terminators, do_sample=True, temperature=0.1, top_p=0.9)
    
    response = outputs[0][input_ids.shape[-1]:]
    reasons_d[message[1]['content'].split("###")[2]] = tokenizer.decode(response, skip_special_tokens=True)

generating CoT: 100%|██████████| 20/20 [04:19<00:00, 12.96s/it]


In [33]:
reasons_d

{' Here is the argument component to be classified: This has gotten worse with twitter, myspace, and facebook': 'Okay, so I need to classify the argument component "This has gotten worse with twitter, myspace, and facebook" as either MajorClaim, Claim, or Premise. Let me break this down step by step.\n\nFirst, I\'ll look at the semantic content. The component is stating that the situation has worsened due to specific platforms. It\'s pointing out a negative impact of technology on language, specifically how social media platforms are contributing to this issue. So, it\'s about the effect of these platforms on language use.\n\nNext, syntactic structure. It\'s a declarative statement, but more importantly, it\'s a specific example supporting the main point. It\'s not just a general statement; it\'s tied to the examples given.\n\nLooking at the context, the essay\'s thesis is that technology designed to simplify life often complicates it. The component is providing evidence that technolog

In [40]:
reasons_l = [v.split("\n\n") for k,v in reasons_d.items()]

In [41]:
reasons_l

[['Okay, so I need to classify the argument component "This has gotten worse with twitter, myspace, and facebook" as either MajorClaim, Claim, or Premise. Let me break this down step by step.',
  "First, I'll look at the semantic content. The component is stating that the situation has worsened due to specific platforms. It's pointing out a negative impact of technology on language, specifically how social media platforms are contributing to this issue. So, it's about the effect of these platforms on language use.",
  "Next, syntactic structure. It's a declarative statement, but more importantly, it's a specific example supporting the main point. It's not just a general statement; it's tied to the examples given.",
  "Looking at the context, the essay's thesis is that technology designed to simplify life often complicates it. The component is providing evidence that technology, specifically social media, is making language worse, which supports the thesis by showing a negative side eff

In [42]:
reasoning_chains = []
classifications = []

for chain in reasons_l:
    reasoning_chains.append(chain[:-1])
    classifications.append(chain[-1])

In [43]:
reasoning_chains

[['Okay, so I need to classify the argument component "This has gotten worse with twitter, myspace, and facebook" as either MajorClaim, Claim, or Premise. Let me break this down step by step.',
  "First, I'll look at the semantic content. The component is stating that the situation has worsened due to specific platforms. It's pointing out a negative impact of technology on language, specifically how social media platforms are contributing to this issue. So, it's about the effect of these platforms on language use.",
  "Next, syntactic structure. It's a declarative statement, but more importantly, it's a specific example supporting the main point. It's not just a general statement; it's tied to the examples given.",
  "Looking at the context, the essay's thesis is that technology designed to simplify life often complicates it. The component is providing evidence that technology, specifically social media, is making language worse, which supports the thesis by showing a negative side eff

In [44]:
classifications

['{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["MajorClaim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Claim"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Premise"]}',
 '{"component_type": ["Claim"]}']

In [46]:
predictions = [json.loads(elem)['component_type'][0] for elem in classifications]

In [47]:
grounds = df_s.label.tolist()

In [48]:
len(grounds), len(predictions)

(20, 20)

In [49]:
predictions

['Premise',
 'Claim',
 'MajorClaim',
 'Claim',
 'Claim',
 'Premise',
 'MajorClaim',
 'MajorClaim',
 'Premise',
 'Claim',
 'Claim',
 'Claim',
 'Claim',
 'Claim',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Premise',
 'Claim']

In [258]:
# input_ids = tokenizer.apply_chat_template(
#     messages,
#     add_generation_prompt=True,
#     return_tensors="pt"
# ).to(model.device) # type: ignore

In [259]:
#input_ids

In [260]:
#print(tokenizer.decode(input_ids[0]))

In [261]:
# terminators = [
#     tokenizer.eos_token_id,
#     tokenizer.convert_tokens_to_ids("<|eot_id|>")
# ]

In [262]:
# outputs = model.generate(
#     input_ids,
#     max_new_tokens=1024,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.1,
#     top_p=0.9,
# )

In [263]:
#response = outputs[0][input_ids.shape[-1]:]

In [264]:
#print(tokenizer.decode(response, skip_special_tokens=True))

In [265]:
#reasons_d[usr_msg] = tokenizer.decode(response, skip_special_tokens=True)

In [266]:
#reasons_d